In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import cartopy.crs as ccrs
import shapely.geometry
from shapely.geometry import MultiPolygon
from skimage import io

In [ ]:
# add datasets
df = gpd.read_file("USA_shapefile/cb_2018_us_state_500k.shp")
df = df.to_crs("ESRI:102009") # changing projection for better appearnce
df2 = gpd.read_file("Image_data/states.csv")

In [ ]:
#rename column to merge 
df2 = df2.rename(columns ={'code':'STUSPS'})
#merging the dataset
df3 = df2.merge(df, on='STUSPS', how = 'outer')
#filtering USA mainland
df3 = df3[df3["STUSPS"].isin(["RI","HI","AK","VI","MP","GU","AS","PR","DC"]) == False]

In [ ]:
#### get the polygon fill maps ####

#setting the plot
plt.figure(figsize=(20, 15),dpi = 100)
ax = plt.axes(projection=ccrs.LambertConformal())
ax.set_extent((-120, -72, 20, 50))
    
#iterating through each row
for index, row in df3.iterrows():     
    
    #reading the image from url 
    image = io.imread(row["skyline_background_url"])
    
    state = row['state']
    state_geometry = row['geometry_y']
    
    #extracting bounding box for each state
    state_bb = list(state_geometry.bounds)
    state_bb[1],state_bb[2] = state_bb[2],state_bb[1]
    
    # transforming geoaxes to matplotlib axes
    plate_carree_transform = ccrs.LambertConformal()._as_mpl_transform(ax)  
   
    
    if state_geometry.geometryType()=='MultiPolygon': ## for ploygons, looping is not required
        for xy in state_geometry.geoms:
            
            #getting the path for creating patches
            coord_xy = xy.exterior.xy  
            xy_list = []
            for xy in zip(coord_xy[0], coord_xy[1]):
                xy_list.append([xy[0],xy[1]])
            
            #plotting images over boudnding boxes
            image2 = ax.imshow(image,extent=state_bb, zorder=5)
            #clipping the images using the patches
            image2.set_clip_path(mpatches.Path(xy_list),transform= plate_carree_transform) 
       
    else:
        #getting the path for creating patches
        coord_xy = state_geometry.exterior.xy   
        xy_list = []
        for xy in zip(coord_xy[0], coord_xy[1]):
            xy_list.append([xy[0],xy[1]])
   
        #plotting images over boudnding boxes
        image2 = ax.imshow(image, extent=state_bb,zorder=5) 
        #clipping the images using the patches
        image2.set_clip_path(mpatches.Path(xy_list),transform=plate_carree_transform) 

    pass

plt.savefig('usa_polygon_fill_map.png')